In [1]:
!pip install git+https://www.github.com/farizrahman4u/eywa.git

  Cloning https://www.github.com/farizrahman4u/eywa.git to /tmp/pip-req-build-5p2u62go
  Running command git clone -q https://www.github.com/farizrahman4u/eywa.git /tmp/pip-req-build-5p2u62go
     |████████████████████████████████| 368kB 5.7MB/s 
     |████████████████████████████████| 655kB 21.7MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 3.3MB 32.8MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 112kB 29.3MB/s 
     |████████████████████████████████| 256kB 42.0MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 3.9MB 55.2MB/s 
     |████████████████████████████████| 112kB 61.1MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 51kB 9.9MB/s 
     |████████████████████████████████

In [2]:
!pip install pyowm

     |████████████████████████████████| 3.8MB 4.4MB/s 


In [3]:
from eywa.nlu import Classifier

CONV_SAMPLES = {
    'greetings' : ['Hi', 'hello', 'How are you', 'hey there', 'hey'],
    'taxi'      : ['book a cab', 'need a ride', 'find me a cab'],
    'weather'   : ['what is the weather in tokyo', 'weather germany',
                   'what is the weather like in kochi',
                   'what is the weather like', 'is it hot outside'],
    'datetime'  : ['what day is today', 'todays date', 'what time is it now',
                   'time now', 'what is the time'],
    'music'     : ['play the Beatles', 'shuffle songs', 'make a sound']
}

CLF = Classifier()
for key in CONV_SAMPLES:
    CLF.fit(CONV_SAMPLES[key], key)

print(CLF.predict('will it rain today'))  # >>> 'weather'
print(CLF.predict('play playlist rock n\'roll'))  # >>> 'music'
print(CLF.predict('what\'s the hour?'))  # >>> 'datetime'

Source: https://github.com/explosion/sense2vec/releases/download/v1.0.0a0/reddit_vectors-1.1.0.tar.gz
Destination /root/.eywa/lang/en/embeddings/reddit_vectors-1.1.0.tar.gz
Size: 560.460747718811MB
586899456/587685689 [============================>.] - 99.866215% - ETA: 0sDone.
Extracting...
Done.
Converting embeddings...
Done.
Seems you are running the program for the first time. Building index...
Building tree...
1193838/1195260 [============================>.] - 99.881030% - ETA: 0sDone.
Creating databases...
1194090/1195260 [============================>.] - 99.902113% - ETA: 0s
Converting words to indices...
1191011/1195260 [============================>.] - 99.644512% - ETA: 0sDone.
datetime
music
weather


In [4]:
from eywa.nlu import EntityExtractor

X_WEATHER = ['what is the weather in tokyo', 'weather germany', 'what is the weather like in kochi']
Y_WEATHER = [{'intent': 'weather', 'place': 'tokyo'}, {'intent': 'weather', 'place': 'germany'},
             {'intent': 'weather', 'place': 'kochi'}]

EX_WEATHER = EntityExtractor()
EX_WEATHER.fit(X_WEATHER, Y_WEATHER)


In [5]:
EX_WEATHER.predict('what is the weather in London')

{'intent': 'weather', 'place': 'London'}

In [6]:
from eywa.nlu import EntityExtractor

x = ['what is the weather in tokyo', 'what is the weather', 'what is the weather like in kochi']
y = [{'intent': 'weather', 'place': 'tokyo'}, {'intent': 'weather', 'place': 'here'}, {'intent': 'weather', 'place': 'kochi'}]

ex = EntityExtractor()
ex.fit(x, y)

x_test = 'what is the weather in london like'
print(ex.predict(x_test))


{'intent': 'weather', 'place': 'like'}


In [18]:
from pyowm import OWM
import logging

LOGGER = logging.getLogger('main')
# put your API key here:
mgr = OWM('API-key')  
# in older versions you had to load OWM().weather_manager() instead

def get_weather_forecast(place):
    LOGGER.warning(place)
    observation = mgr.weather_at_place(place)
    return observation.get_weather().get_detailed_status()

print(get_weather_forecast('London'))

London


broken clouds


In [8]:
X_GREETING = ['Hii', 'helllo', 'Howdy', 'hey there', 'hey', 'Hi']
Y_GREETING = [{'greet': 'Hii'}, {'greet': 'helllo'}, {'greet': 'Howdy'},
              {'greet': 'hey'}, {'greet': 'hey'}, {'greet': 'Hi'}]

EX_GREETING = EntityExtractor()
EX_GREETING.fit(X_GREETING, Y_GREETING)

In [9]:
X_DATETIME = ['what day is today', 'date today', 'what time is it now', 'time now']
Y_DATETIME = [{'intent' : 'day', 'target': 'today'}, {'intent' : 'date', 'target': 'today'},
              {'intent' : 'time', 'target': 'now'}, {'intent' : 'time', 'target': 'now'}]

EX_DATETIME = EntityExtractor()
EX_DATETIME.fit(X_DATETIME, Y_DATETIME)

In [10]:
import datetime

_EXTRACTORS = {
  'taxi': None,
  'weather': EX_WEATHER,
  'greetings': EX_GREETING,
  'datetime': EX_DATETIME,
  'music': None
}

def question_and_answer(u_query):
    '''Answer a user query
    '''
    q_class = CLF.predict(u_query)
    if _EXTRACTORS[q_class] is None:
      return 'Sorry, you have to upgrade your software!'

    q_entities = _EXTRACTORS[q_class].predict(u_query)
    if q_class == 'greetings':
      return q_entities.get('greet', 'hello')
    
    if q_class == 'weather':
      place = q_entities.get('place', 'London').replace('_', ' ')
      return 'The forecast for {} is {}'.format(
          place,
          get_weather_forecast(place)
      )

    if q_class == 'datetime':
      return 'Today\'s date is {}'.format(
          datetime.datetime.today().strftime('%B %d, %Y')
      )
    
    return 'I couldn\'t understand what you said. I am sorry.'

In [ ]:
while True:
  query = input('\nHow can I help you? ')
  print(question_and_answer(query))

germany


The forecast for germany is light rain
